In [1]:
%load_ext pycodestyle_magic
%flake8_on

In [2]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from torchvision.datasets import MNIST
import torch.nn.functional as F

In [3]:
trainset = MNIST('../', download=True, train=True)
testset = MNIST('../', download=True, train=False)

y_trainset = trainset.targets
y_testset = testset.targets

# this time, we resize the data to have directly one channel for convolutions
trainset = trainset.data.reshape(60000, 1, 28, 28).to(torch.float32)
testset = testset.data.reshape(10000, 1, 28, 28).to(torch.float32)

# normalize
m, s = trainset.mean(), trainset.std()
trainset = (trainset - m) / s
testset = (testset - m) / s

# Import (not define) useful classes
so far we've been defining Dataset, DataLoader and Optimizer. That was mainly an exercise to recreate them from scratch and get a deep understanding of what they do exactly. But we can now import them from Pytorch

In [4]:
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split

from torch.optim import SGD

In [5]:
def accuracy(output, target):
    return (torch.argmax(output, dim=1) == target).float().mean()

In [6]:
# Still, we've got to create our own Dataset Class inheriting from Dataset
class MNIST_Dataset(Dataset):
    def __init__(self, x_tensor, y_tensor):
        self.x = x_tensor
        self.y = y_tensor

    def __getitem__(self, idx):
        return (self.x[idx], self.y[idx])

    def __len__(self):
        return len(self.x)

In [7]:
x_train, x_valid = trainset[0:50000, :], trainset[50000:, :]
y_train, y_valid = y_trainset[0:50000], y_trainset[50000:]

train = MNIST_Dataset(x_train, y_train)
valid = MNIST_Dataset(x_valid, y_valid)

In [8]:
EPOCHS = 5
bs = 64
lr = 0.05
loss_func = F.cross_entropy


train_dl = DataLoader(train, bs, shuffle=True)
valid_dl = DataLoader(valid, bs, shuffle=False)

In [156]:
# Define model


class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func

    def forward(self, x):
        return self.func(x)


def flatten(x):
    return x.view(x.shape[0], -1)


model = nn.Sequential(
    nn.Conv2d(in_channels=1, out_channels=4,
              kernel_size=3, stride=2, padding=1),  # bs*4*14*14
    nn.ReLU(),
    nn.Conv2d(4, 8, 3, 2, 1),  # bs * 8 * 7 * 7
    nn.ReLU(),
    nn.Conv2d(8, 16, 3, 2, 1),  # bs * 16 * 4 * 4
    nn.ReLU(),
    nn.Conv2d(16, 32, 3, 2, 1),  # bs * 32 * 2 * 2
    nn.ReLU(),
    nn.Conv2d(32, 64, 3, 2, 1),  # bs * 64 * 1 * 1
    nn.AdaptiveAvgPool2d(1),
    Lambda(flatten),
    nn.Linear(64, 10)
)

opt = SGD(model.parameters(), lr)

In [157]:
for epoch in range(EPOCHS):
    model.train()
    for xb, yb in train_dl:
        loss = loss_func(model.cuda()(xb.cuda()), yb.cuda())
        loss.backward()
        opt.step()
        opt.zero_grad()

    model.eval()
    with torch.no_grad():
        total_loss, total_acc = 0., 0.
        for xb, yb in valid_dl:
            pred = model(xb.cuda())
            total_loss += loss_func(pred, yb.cuda())
            total_acc += accuracy(pred, yb.cuda())
        n_entries = len(valid_dl)
        print('epoch', epoch,
              'loss:', (total_loss/n_entries).item(),
              'accuracy:', (total_acc/n_entries).item()
              )

epoch 0 loss: 0.5748570561408997 accuracy: 0.8170780539512634
epoch 1 loss: 0.20386554300785065 accuracy: 0.9351114630699158
epoch 2 loss: 0.12395601719617844 accuracy: 0.9615843892097473
epoch 3 loss: 0.16629882156848907 accuracy: 0.9493431448936462
epoch 4 loss: 0.15520162880420685 accuracy: 0.9537221193313599


# Batch normalization 
<a href='https://arxiv.org/pdf/1502.03167.pdf'> link towards the paper </a><br />
The loss varies a lot. This is because we haven't introduced batch normalization into the model. Batch Normalization forces the activations to have unit variance (i.e mean 0 and standard deviation 1). This is important because, since every time we backpropagate the previous layer change, its output distribution will change as well. 

Quote from the paper:

<blockquote>the inputs to each layerare affected by the parameters of all preceding layers – sothat small changes to the network parameters amplify as the network becomes deeper.</blockquote>

So the next layer is used to receive numbers with a certain distribution, and suddenly this distribution change! How is this layer supposed to learn which parameters it should have to perform the multiplication with its input, if the input is never similar ?

This is why we want the input distribution, at every layer, to be, if not identically, at least similarly distributed.

<blockquote>As such it is advantageous for thedistribution of x to remain fixed over time. Then, Θ2 does 
not have to readjust to compensate for the change in the distribution of x. </blockquote>



## How do we do that ?

<img src='data/images/BN algorithm.png'>

In [16]:
# For each minibatch:
#     take its mean
#     take its variance
#     use both to normalize the inputs
#     scale and shift using Gamma and Beta parameters

In [61]:
# nn.BatchNorm2d??  # result: source code next cell

# Init signature:
# nn.BatchNorm2d(
#     num_features,
#     eps=1e-05,
#     momentum=0.1,
#     affine=True,
#     track_running_stats=True,
# )

Source code:

```python
Source:        
class BatchNorm2d(_BatchNorm):
    Applies Batch Normalization over a 4D input (a mini-batch of 2D inputs
    with additional channel dimension) as described in the paper
    `Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift`
```
\begin{align*}
        y = \frac{x - \mathrm{E}[x]}{ \sqrt{\mathrm{Var}[x] + \epsilon}} * \gamma + \beta
\end{align*}

The mean and standard-deviation are calculated per-dimension over
the mini-batches and $ `\gamma` $ and $ `\beta` $  are learnable parameter vectors
of size `C` (where `C` is the input size). By default, the elements of $ `\gamma` $  are set
to 1 and the elements of $ `\beta` $  are set to 0.

In [21]:
class BatchNorm(nn.Module):
    def __init__(self, nf, mom=0.1, eps=1e-5):
        super().__init__()
        self.mom, self.eps = mom, eps
        # in the paper, gamma and beta are 'parameters to be learned'
        self.gamma = nn.Parameter(torch.ones(nf, 1, 1))
        self.beta = nn.Parameter(torch.zeros(nf, 1, 1))

        # use register_buffer when you want a stateful part of your model
        # that is not a parameter, but you want it in your state_dict
        # Here, we want to keep vars and means for latter inference,
        # so we register, but they are not supposed to be affected by
        # back prop (they are not parameter: we can't use register_parameter)
        self.register_buffer('vars', torch.ones(1, nf, 1, 1))
        self.register_buffer('means', torch.zeros(1, nf, 1, 1))

    def update_stats(self, x):
        # mean((0,2,3)) means "do a different mean for each channel"
        m = x.mean((0, 2, 3), keepdim=True)
        v = x.var((0, 2, 3), keepdim=True)
        # we use lerp_ as a proxy for moving average
        self.means.lerp_(m, self.mom)
        self.vars.lerp_(v, self.mom)
        return m, v

    def forward(self, x):
        if self.training:
            with torch.no_grad():
                m, v = self.update_stats(x)
        else:
            m, v = self.means, self.vars
        x = (x-m) / (v+self.eps).sqrt()
        return x*self.gamma + self.beta

In [161]:
# This is starting to be messy. We'll see if we can factor this out
model = nn.Sequential(
    nn.Conv2d(in_channels=1, out_channels=8,
              kernel_size=3, stride=2, padding=1, bias=False),  # bs*4*14*14
    BatchNorm(8),
    nn.ReLU(),
    nn.Conv2d(8, 16, 3, 2, 1, bias=False),  # bs * 8 * 7 * 7
    BatchNorm(16),
    nn.ReLU(),
    nn.Conv2d(16, 32, 3, 2, 1, bias=False),  # bs * 16 * 4 * 4
    BatchNorm(32),
    nn.ReLU(),
    nn.Conv2d(32, 64, 3, 2, 1, bias=False),  # bs * 32 * 2 * 2
    BatchNorm(64),
    nn.ReLU(),
    nn.Conv2d(64, 64, 3, 2, 1, bias=False),  # bs * 64 * 1 * 1
    BatchNorm(64),
    nn.AdaptiveAvgPool2d(1),
    Lambda(flatten),
    nn.Linear(64, 10)
)

Should we put Batchnorm after or before relu ?
According to the paper, before, but it appears no careful experiment has been done to figure this out:
https://forums.fast.ai/t/where-should-i-place-the-batch-normalization-layer-s/56825/4

In [163]:
# Note for future review: at first I had forgotten to set a new optimizer.
# As a result, model didn't learn anything.
# That's because SGD was then optimizing parameters that didn't exist anymore
opt = SGD(model.parameters(), lr)

In [164]:
for epoch in range(EPOCHS):
    model.train()
    for xb, yb in train_dl:
        loss = loss_func(model.cuda()(xb.cuda()), yb.cuda())
        loss.backward()
        opt.step()
        opt.zero_grad()

    model.eval()
    with torch.no_grad():
        total_loss, total_acc = 0., 0.
        for xb, yb in valid_dl:
            pred = model(xb.cuda())
            total_loss += loss_func(pred, yb.cuda())
            total_acc += accuracy(pred, yb.cuda())
        n_entries = len(valid_dl)
        print('epoch', epoch,
              'loss:', (total_loss/n_entries).item(),
              'accuracy:', (total_acc/n_entries).item()
              )

epoch 0 loss: 0.06602656841278076 accuracy: 0.9791998267173767
epoch 1 loss: 0.05816727131605148 accuracy: 0.9821855425834656
epoch 2 loss: 0.060302749276161194 accuracy: 0.9821855425834656
epoch 3 loss: 0.057873792946338654 accuracy: 0.9835788607597351
epoch 4 loss: 0.05947509780526161 accuracy: 0.9819864630699158


First of all, the loss is less bumpy. Second: the accuracy jumps to 0.98 ! 

However, now the model definition is starting to get huge, and each time I want to change the number of filters I have to manually change it through all the layers. That's error-prone and time-consuming. 

Also, the training loop with validation added is once again messy as well. In the next notebook we'll refactor this out.